In [1]:
import torch
import pickle

from encoding_utils import *
from flatten_lattice import *
from model_construct import *

2022-09-10 12:22:09.615618: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-10 12:22:09.615652: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def submask_help(words):
    msk = []
    for i in range(len(words)-1):
        if "#" in words[i+1]:
            msk.append(0)
        else:
            msk.append(1)
    msk.append(1)
    return torch.tensor(msk)

def subword_mask_all ():
    msk = torch.ones_like(sents)
    for i in range(0, len(sents)):
        tmp = [tok.decode(s) for s in sents[i]]
        msk[i] = submask_help(tmp)
    return msk

def get_acc():
    # simplify prediction tensors
    ysimp = ylabels
    psimp = torch.argmax(pred1, dim=2)
    # clean up labels
    sm = subword_mask_all()
    ysimp[sents==0] = 0
    ysimp[sents==102] = 0
    ysimp[sm==0] = 0
    ysimp[:, 0] = 0
    # apply cleanaup to x 
    psimp[ysimp==0] = 0
    # apply cleanaup to x 
    psimp[ysimp==0] = 0
    # compute accuracy
    acc = 1 - ((ysimp-psimp).count_nonzero())/ysimp.count_nonzero()
    return acc, ysimp, psimp

def get_err_tensors():
    diff = ysimp-psimp
    diff = diff.abs().bool().int()
    errg = ysimp
    errp = psimp
    errg[diff==0] = 0
    errp[diff==0] = 0
    return errg, errp


def print_decoded(toks):
    nt = []
    for t in toks:
        if t==100 or t==101 or t==102 or t==0:
            continue
        nt.append(t)
    print(tok.decode(nt))
    return tok.decode(nt)
    
# 
def error_vis(ind):
    nt = print_decoded(sents[ind])
    p = [lablist[int(errp[ind][i])] for i in errp[ind].nonzero()]
    g = [lablist[int(errg[ind][i])] for i in errp[ind].nonzero()]
    s = [tok.decode(int(sents[ind][i])) for i in errp[ind].nonzero()]
    print("ERRORS : ")
    print("(predicted, gold, token)")
    print(list(zip(p, g, s)))
    return nt
    
def vis_model_pred(strinp):
    tokinps = tok(strinp, return_tensors='pt').input_ids.to(device)
    msk = torch.tril(torch.ones((len(tokinps[0]), len(tokinps[0]))))
    preds = bmodel(tokinps, attmasks=torch.stack([msk]).to(device))
    preds = torch.argmax(preds, dim=2)
    labs = [lablist[int(p)] for p in preds[0]]
    tinps = [tok.decode(int(t))for t in tokinps[0]]
    #print(list(zip(labs, tinps)))

In [3]:
tok = fl.bert_tok

# loads in odata with keys: tmaps, masks, pgraphs
with open('torchsaved/outputv5.pkl', 'rb') as file:
    odata = pickle.load(file)
    
with open('./a3distrib/lab_vocab.json') as json_file:
    labels = json.load(json_file)
    
# load in other metadata
bmodel = load_model(labels)
sents, posids = create_inputs(odata['pgraphs'])

ylabels = tmap_pos_goldlabels(odata['tmaps'], sents)   
lablist = [l for l in labels.keys()]
torch.cuda.empty_cache()

pg = odata['pgraphs']

sents, posids = create_inputs(odata['pgraphs'])
pred1 = bmodel(sents, mod_posids(posids), odata['masks'])
accuracy, ysimp, psimp = get_acc()

print("accuracy: ", accuracy)

errg, errp = get_err_tensors()

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


GPU Mem Used =  534572544
accuracy:  tensor(0.8894, device='cuda:0')


In [4]:
# can put any number between 0-100 and it will show an example and errors with the example
INP = 0
vis_model_pred(error_vis(1))

Previous vaccines against tobacco had been ineffective because of antibodies.
ERRORS : 
(predicted, gold, token)
[('JJ', 'IN', 'Previous'), ('.', 'IN', '.')]


In [25]:
#################### BEYOND THIS IS RANDOM SANITY CHECKING ########################################

In [ ]:
goldlist = []
predlist = []
toklist = []
for e in errg.nonzero():
    goldlist.append(lablist[int(errg[e[0], e[1]])])
    predlist.append(lablist[int(errp[e[0], e[1]])])
    toklist.append(tok.decode(int(sents[e[0], e[1]])))
    
zipped = list(zip(goldlist, predlist, toklist))
zipcln = []
cnt = 0
for z in zipped:
    if z[1]==z[2]:
        cnt+=1
        continue
    zipcln.append(z)
    

In [ ]:
# sanity check masking 
sents.shape

In [19]:
odata['masks'].shape

torch.Size([101, 500, 500])

In [29]:
def print_mask_leadup(ind):
    msk = odata['masks'][ind]
    smat = torch.zeros_like(msk)
    smat[:, :] = sents[ind]
    assert msk.shape == smat.shape
    return smat, msk

In [30]:
sm, m = print_mask_leadup(0)

In [33]:
sm[m==0]=0

In [40]:
inps = tok.batch_decode(sm.int())

In [42]:
inps = [txt.replace(" [PAD]", "") for txt in inps]

In [45]:
posids[0]

tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,
         29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  21,  22,  23,
         24,  25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,
         38,  16,  17,  18,  19,  20,  21,  22,  23,  22,  20,  21,  21,  22,
         23,  24,  21,  22,  23,  24,  25,  26,  27,  28,  31,  22,  23,  24,
         25,  27,  28,  22,  18,  19,  20,  21,  22,  22,  19,  20,  21,  22,
         22,  23,  24,  25,  22,  23,  24,  25,  23,  24,  25,  26,  27,  28,
         29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  26,
         27,  28,  29,  30,  31,  32,  33,  28,  29,  30,  31,  32,  35,  33,
         34,  36,  34,  34,  28,  23,  24,  24,  25,  26,  27,  28,  29,  26,
         27,  28,  26,  26,  27,  27,  28,  29,  32,  29,  30,  31,  24,  25,
         26,  25,  26,  27,  28,  31,  32,  33,  34,  35,  36,  

In [51]:
a = torch.zeros(5, 5)
a[:3, :3] = torch.ones(3, 3)

In [54]:
torch.tril(a)

tensor([[1., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])